In [1]:
%load_ext jupyter_cms

In [2]:
import os
from Bio import SeqIO

In [3]:
#here is the in file defined containing alternative 
file_path ='/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/'
file_name='v91_pa_ctg.fa'
bash_script_q="v91_pa_ctg_qmapping.sh"
bash_script_g="v91_pa_ctg_ctg_gmapping.sh"
outdir = '/Users/ben/Documents/work_related/Canberra/RJ_lab_projects/Rust_genome_sequencing/PST79/analysis/PacBio_SCH1743/FALCON_0_4_assemblies/p_assemblies/v9_1/v91_pa/nucmer_h_on_pwh/'

In [7]:
total_seq_recs = 0
p_contigs =[]
a_contigs = []
a_count =0
p_count =0
for seq_record in SeqIO.parse(file_path+'/'+file_name, "fasta"):
    if "-" in list(seq_record.id):
        a_contigs.append(seq_record)
        a_count += 1
    else:
        p_contigs.append(seq_record)
        p_count += 1
    total_seq_recs += 1
total_seq_recs == p_count + a_count

True

In [8]:
os.chdir(outdir)

In [9]:
SeqIO.write(a_contigs,file_name[:-3]+'_a_ctg.fa' , 'fasta')
SeqIO.write(p_contigs,file_name[:-3]+'_p_ctg.fa' , 'fasta')

297

In [10]:
!ls

v91_pa_ctg_a_ctg.fa v91_pa_ctg_p_ctg.fa


In [11]:
outfq = open(bash_script_q, 'w')
outfq.write('#!/bin/bash\n')
outfg = open(bash_script_g, 'w')
outfg.write('#!/bin/bash\n')
for pseq_record in SeqIO.parse(file_name[:-3]+'_p_ctg.fa', 'fasta'):
    p_acontigs = []
    p_contig = pseq_record
    suffix = file_name[:-3]+pseq_record.id.split("|")[0]+"pap"
    p_file = file_name[0:-3]+"_"+pseq_record.id.split("|")[0]+'.fa'
    SeqIO.write(p_contig, p_file, 'fasta')
    for aseq_record in SeqIO.parse(file_name[:-3]+'_a_ctg.fa', 'fasta'):
        if aseq_record.id.split("|")[0].split("_")[0] == pseq_record.id.split("|")[0]:
            p_acontigs.append(aseq_record)
    a_file = file_name[:-3]+"_"+pseq_record.id.split("|")[0]+'_a_ctg.fa'
    #if we have alternative contigs safe those too
    if p_acontigs != []:
        SeqIO.write(p_acontigs, a_file, 'fasta')
        outfq.write('nucmer '+p_file+' '+a_file+" > "+'out.delta\n')
        outfq.write('delta-filter -q '+'out.delta'+" > "+suffix+"_qfiltered.delta\n")
        outfq.write('show-coords -T '+suffix+"_qfiltered.delta > "+suffix+".qcoords\n")
        outfq.write('mummerplot -p '+suffix+'_qfiltered --postscript '+suffix+"_qfiltered.delta\n")
        outfq.write('mummerplot -c -p '+suffix+'_qfiltered_cov --postscript '+suffix+"_qfiltered.delta\n")
        #for g_file bash script
        outfg.write('nucmer '+p_file+' '+a_file+" > "+'out.delta\n')
        outfg.write('delta-filter -g '+'out.delta'+" > "+suffix+"_gfiltered.delta\n")
        outfg.write('show-coords -T '+suffix+"_gfiltered.delta > "+suffix+".gcoords\n")
        outfg.write('mummerplot -p '+suffix+'_gfiltered --postscript '+suffix+"_gfiltered.delta\n")
        outfg.write('mummerplot -c -p  '+suffix+'_gfiltered_cov --postscript '+suffix+"_gfiltered.delta\n")
outfq.close()
outfg.close()